In [1]:
import sys  
sys.path.insert(0, '../')

import wandb
import os
import re

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

sns.set(style='darkgrid')
%matplotlib inline

import numpy as np
import cupy as cp
import scipy
import model.functions as f
from chainer import serializers
from chainer.backends.cuda import to_cpu, to_gpu

In [2]:
class Bunch(object):
    def __init__(self, adict):
        self.__dict__.update(adict)

def cos_similarity(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

gpu_id = 0

In [3]:
name = 'mnist_fc_none_20000_0'

# check id
api = wandb.Api()
runs = api.runs("liuyuezhang/cerebellum")
for run in runs:
    if run.name == name:
        run_id = run.id
        config = Bunch(run.config)
        print(run.name, run.id)
        break

mnist_fc_none_20000_0 2s5rhpwv


In [4]:
from data.mini_mnist import get_mini_mnist
from chainer.datasets import get_mnist, get_cifar10

# data
if config.env == 'mnist1':
    data = get_mini_mnist(c=1)
    in_size = 28 * 28
    out_size = 1
elif config.env == 'mnist2':
    data = get_mini_mnist(c=2)
    in_size = 28 * 28
    out_size = 2
elif config.env == 'mnist':
    data = get_mnist(withlabel=True, ndim=1)[1]
    in_size = 28 * 28
    out_size = 10
else:
    raise NotImplementedError

X, Y = zip(*data)
X = np.array(X).T
Y = f.one_hot(Y, out_size=out_size, dtype=float).T
X.shape, Y.shape

((784, 10000), (10, 10000))

In [5]:
from model.cerebellum import Cerebellum
model = Cerebellum(in_size=in_size, out_size=out_size, args=config).to_gpu(gpu_id)

# load
for file in os.listdir('../wandb'):
    if run_id in file:
        serializers.load_npz('../wandb' + '/' + file + '/model.pkl', model)
        print("load model")
        break

load model


In [6]:
Z = model.project(to_gpu(X.T)).array.T
W_model = model.purkinje.W.array

pred_model = W_model @ Z

Z.shape, W_model.shape

((20000, 10000), (10, 20000))

In [76]:
np.sum(pred_model.argmax(0) == Y.argmax(0))

9505

In [77]:
W = Y @ Z.T @ np.linalg.inv(Z @ Z.T)
W.shape

(10, 2000)

In [78]:
pred = W @ Z
np.sum(pred.argmax(0) == Y.argmax(0))

9906

In [81]:
u = W[0]
v = W_model[0]
cos_similarity(u, v)

0.6075046534464619

In [10]:
calc_dim(Z)

array(35.518925, dtype=float32)

In [7]:
def calc_dim(X):
    C = X @ X.T
    return cp.trace(C) ** 2 / cp.trace(C @ C)

In [9]:
def calc_dim(X):
    X = X - cp.mean(X, axis=1).reshape(-1, 1)
    C = X @ X.T
    return cp.trace(C) ** 2 / cp.trace(C @ C)